In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
ROOT = '/content/drive/MyDrive/Colab Notebooks/coursework2'
TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4ODc3MTlkYS1mMzgzLTQzM2ItYTg5OC1jOGU3MTNmOWZhMjAifQ=='

In [3]:
# !unzip -u '/content/drive/MyDrive/Colab Notebooks/coursework2/faster_rcnn.zip' -d '/content/drive/MyDrive/Colab Notebooks/coursework2/'

In [4]:
import sys
sys.path.insert(0, ROOT)

In [81]:
!pip install einops
!pip install --upgrade --quiet neptune-client
!pip install pytorch-lightning

In [82]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import nn
from RTSD import RTSD
from common import *
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import matplotlib.patches as patches
import tqdm
import neptune.new as neptune
from pytorch_lightning.loggers.neptune import NeptuneLogger
from model50 import *
from model18 import *
from pytorch_lightning.callbacks import ModelCheckpoint
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Training model with resnet50 as backbone and SGD optimizer**

In [89]:
MODEL_NAME = "v1"
PARAMS = {
    'optimizer_params':
      {
        'momentum': 0.9,
        'weight_decay': 0.0005
      },
    'learning_rate': 0.005,
    'num_classes': 198,
    'sys/name': MODEL_NAME,
    'sys/tags': ['resnet50', 'fpn', 'SGD']
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    upload_source_files=[f'{ROOT}/model50.py', f'{ROOT}/train.ipynb'])

model = LitModel50(
                 optimizer=torch.optim.SGD,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

data_module = RTSD_DataModule(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',

)

NeptuneLogger will work in online mode


                          filename  x_from  y_from  ...  height  sign_class sign_id
0  autosave01_02_2012_09_13_33.jpg     649     376  ...      18         2_1       0
1  autosave01_02_2012_09_13_34.jpg     671     356  ...      21         2_1       0
2  autosave01_02_2012_09_13_35.jpg     711     332  ...      26         2_1       0
3  autosave01_02_2012_09_13_36.jpg     764     290  ...      36         2_1       0
4  autosave01_02_2012_09_13_36.jpg     684     384  ...      17        1_23       1

[5 rows x 7 columns]
59188
59188
cnt = 0
53269
5919


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [90]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/sosa/coursework2/e/COUR-26



  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 42.3 M
-------------------------------------
42.1 M    Trainable params
222 K     Non-trainable params
42.3 M    Total params
169.215   Total estimated model params size (MB)


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/threads/ping_thread.py", line 37, in run
    self.__backend.ping_experiment(self.__experiment)
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/api_clients/hosted_api_clients/hosted_alpha_leaderboard_api_client.py", line 391, in ping_experiment
    self.leaderboard_swagger_client.api.ping(experimentId=str(experiment.internal_id)).response().result
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 239, in response
    six.reraise(*sys.exc_info())
  File "/usr/local/lib/python3.7/dist-packages/six.py", line 703, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 124, in wrapper
    return func(self, *args, **

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  8547.2         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8539.7         	|1              	|  8539.7         	|  99.913         	|
run_training_batch                 	|  1.2809         	|6659           	|  8529.4         	|  99.791         	|
optimizer_step_and_closure_0       	|  0.70301        	|6659           	|  4681.4         	|  54.771         	|
training_step_and_backward         	|  0.69866        	|6659           	|  4652.4         	|  54.432         	|
model_forward                      

**Training and testing model with ADAM optimizer**

In [76]:
MODEL_NAME = "v1"
PARAMS = {
    'optimizer_params': {
        'weight_decay': 0.001
    },
    'learning_rate': 0.0001,
    'num_classes': 198,   
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    name=MODEL_NAME,
    tags=['resnet50', 'fpn', 'ADAM', 'augment'],
    upload_source_files=[f'{ROOT}/model150.py', f'{ROOT}/train.ipynb']
    )

model = LitModel(
                 optimizer=torch.optim.Adam,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

data_module = RTSD_DataModule(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',

)

NeptuneLogger will work in online mode


NameError: ignored

In [ ]:
trainer.fit(model, data_module)

                          filename  x_from  y_from  ...  height  sign_class sign_id
0  autosave01_02_2012_09_13_33.jpg     649     376  ...      18         2_1       0
1  autosave01_02_2012_09_13_34.jpg     671     356  ...      21         2_1       0
2  autosave01_02_2012_09_13_35.jpg     711     332  ...      26         2_1       0
3  autosave01_02_2012_09_13_36.jpg     764     290  ...      36         2_1       0
4  autosave01_02_2012_09_13_36.jpg     684     384  ...      17        1_23       1

[5 rows x 7 columns]
59188
59188
cnt = 0
53269
5919


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/sosa/coursework2/e/COUR-10



  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 42.3 M
-------------------------------------
42.1 M    Trainable params
222 K     Non-trainable params
42.3 M    Total params
169.215   Total estimated model params size (MB)


Unexpected error in ping thread.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/threads/ping_thread.py", line 37, in run
    self.__backend.ping_experiment(self.__experiment)
  File "/usr/local/lib/python3.7/dist-packages/neptune/internal/api_clients/hosted_api_clients/hosted_alpha_leaderboard_api_client.py", line 391, in ping_experiment
    self.leaderboard_swagger_client.api.ping(experimentId=str(experiment.internal_id)).response().result
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 239, in response
    six.reraise(*sys.exc_info())
  File "/usr/local/lib/python3.7/dist-packages/six.py", line 703, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 200, in response
    swagger_result = self._get_swagger_result(incoming_response)
  File "/usr/local/lib/python3.7/dist-packages/bravado/http_future.py", line 124, in wrapper
    return func(self, *args, **

Saved model!



FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  8646.4         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8608.9         	|1              	|  8608.9         	|  99.565         	|
run_training_batch                 	|  1.2903         	|6659           	|  8592.0         	|  99.37          	|
optimizer_step_and_closure_0       	|  0.99318        	|6659           	|  6613.6         	|  76.489         	|
training_step_and_backward         	|  0.70222        	|6659           	|  4676.1         	|  54.081         	|
model_forward                      

In [ ]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  warnings.warn(*args, **kwargs)


TEST Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
---------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  9171.3         	|  100 %          	|
---------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  8608.9         	|1              	|  8608.9         	|  93.868         	|
run_training_batch                 	|  1.2903         	|6659           	|  8592.0         	|  93.683         	|
optimizer_step_and_closure_0       	|  0.99318        	|6659           	|  6613.6         	|  72.112         	|
training_step_and_backward         	|  0.70222        	|6659           	|  4676.1         	|  50.986         	|
model_forward                   


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{}
--------------------------------------------------------------------------------


[{}]

**Adam optimizer показал результаты хуже SGD ( не зря во всех статьях используют SGD)**

**Training model with resnet18 as backbone and augment**

In [71]:
checkpoint_callback = ModelCheckpoint(
                                      dirpath=f'{ROOT}/lightning_checkpoints/resnet18/epoch=0-step=6658.ckpt'
                                      )

In [72]:
MODEL_NAME = "v2"
PARAMS = {
    'optimizer_params':
      {
        'momentum': 0.9,
        'weight_decay': 0.0005
      },
    'learning_rate': 0.001,
    'num_classes': 198,
    'sys/name': MODEL_NAME,
    'sys/tags': ['resnet18', 'fpn', 'SGD', 'augment']
}

neptune_logger = NeptuneLogger(
    api_key=TOKEN,
    project_name='sosa/coursework2',
    params=PARAMS,
    upload_source_files=[f'{ROOT}/model18.py', f'{ROOT}/train.ipynb'])

model = LitModel18(
                 optimizer=torch.optim.SGD,
                 learning_rate = PARAMS['learning_rate'],
                 optimizer_params = PARAMS['optimizer_params'],
                 root = ROOT
                )

'''model = LitModel18.load_from_checkpoint(f'{ROOT}/lightning_checkpoints/resnet18/epoch=0-step=6658.ckpt',
                                                         optimizer=torch.optim.SGD,
                                          learning_rate = PARAMS['learning_rate'],
                                          optimizer_params = PARAMS['optimizer_params'],
                                          root = ROOT
                                        )'''


data_module = RTSD_DataModule_Augment(
    batch_size = 8,
    root = ROOT
)

trainer = pl.Trainer(
    gpus=-1,
    max_epochs=5,
    logger = neptune_logger,
    profiler='simple',
    callbacks=[checkpoint_callback]
)

NeptuneLogger will work in online mode


                          filename  x_from  y_from  ...  height  sign_class sign_id
0  autosave01_02_2012_09_13_33.jpg     649     376  ...      18         2_1       0
1  autosave01_02_2012_09_13_34.jpg     671     356  ...      21         2_1       0
2  autosave01_02_2012_09_13_35.jpg     711     332  ...      26         2_1       0
3  autosave01_02_2012_09_13_36.jpg     764     290  ...      36         2_1       0
4  autosave01_02_2012_09_13_36.jpg     684     384  ...      17        1_23       1

[5 rows x 7 columns]
59188
59188
cnt = 0
53269
5919


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [74]:
test_model(f'{ROOT}/models/model18_epoch0.pt',
           data_module.test_dataloader())

740it [03:02,  4.05it/s]

Final:
Recall = 0.10369932432432426
Precision = 0.16334553303303312


(0.16334553303303312, 0.10369932432432426, 0.1268612440212973)

In [73]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.3 M    Trainable params
0         Non-trainable params
41.3 M    Total params
165.321   Total estimated model params size (MB)


https://app.neptune.ai/sosa/coursework2/e/COUR-24
Saved model!


FileExistsError: ignored

In [ ]:

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
num_classes = 198
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

checkpoint = torch.load(f'{ROOT}/models/rcnn_v1.pt', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer.state_dict'])
start_epoch = checkpoint['epoch'] + 1

print(f"Loaded model's checkpoint! Start epoch = {start_epoch}")

Loaded model's checkpoint! Start epoch = 1


In [ ]:
model.eval()
recall = 0
precision = 0
batches = 0
for index, (images, targets) in tqdm.tqdm(enumerate(test_loader)):
    images = [img.to(device) for img in images]
    targets = [trgt for trgt in targets]
    for trgt in targets:
        trgt['boxes'] = trgt['boxes'].to(device)
        trgt['labels'] = trgt['labels'].to(device)
    bboxes = model(images)
    batch = [images]
    batch.append(bboxes)
    batch = intersect(batch)
    recall += get_recall(batch[1], targets)
    precision += get_precision(batch[1], targets)
    batches += 1

    #plot_images_bb(batch, targets)
    if index % 100 == 0:
      print()
      print(f'Recall = {recall / batches}')
      print(f'Precision = {precision / batches}')

recall /= batches
precision /= batches
print("Final:")
print(f'Recall = {recall}')
print(f'Precision = {precision}')
run['precision'] = precision
run['recall'] = recall
run['f1'] = 2 * precision * recall / (precision + recall)




0it [00:00, ?it/s]


1it [00:02,  2.84s/it]


Recall = 0.625
Precision = 0.625





2it [00:04,  2.52s/it]


3it [00:06,  2.47s/it]


4it [00:08,  2.27s/it]


5it [00:11,  2.30s/it]


6it [00:13,  2.31s/it]


7it [00:16,  2.40s/it]


8it [00:18,  2.54s/it]


9it [00:21,  2.40s/it]


10it [00:24,  2.64s/it]


11it [00:26,  2.60s/it]


12it [00:30,  2.83s/it]


13it [00:32,  2.57s/it]


14it [00:35,  2.77s/it]


15it [00:37,  2.67s/it]


16it [00:40,  2.72s/it]


17it [00:43,  2.63s/it]


18it [00:44,  2.23s/it]


19it [00:46,  2.09s/it]


20it [00:49,  2.36s/it]


21it [00:50,  2.15s/it]


22it [00:53,  2.24s/it]


23it [00:55,  2.33s/it]


24it [00:57,  2.14s/it]


25it [00:59,  1.99s/it]


26it [01:01,  2.15s/it]


27it [01:04,  2.25s/it]


28it [01:06,  2.37s/it]


29it [01:08,  2.24s/it]


30it [01:10,  1.99s/it]


31it [01:12,  2.01s/it]


32it [01:15,  2.31s/it]


33it [01:17,  2.22s/it]


34it [01:18,  2.04s/it]


35it [01:21,  2.23s/it]


36it [01:23,  2.20s/it]


37it [01:25,  2.12s/it]


38it [01:27,  2.04s/it]


39it [01:29,  2.02s/it]


40it [01:30,  1.8


Recall = 0.6539235619990571
Precision = 0.6614318128241397





102it [03:55,  2.43s/it]


103it [03:57,  2.24s/it]


104it [04:00,  2.40s/it]


105it [04:02,  2.36s/it]


106it [04:05,  2.47s/it]


107it [04:07,  2.48s/it]


108it [04:10,  2.54s/it]


109it [04:13,  2.55s/it]


110it [04:15,  2.61s/it]


111it [04:18,  2.48s/it]


112it [04:20,  2.45s/it]


113it [04:23,  2.49s/it]


114it [04:26,  2.68s/it]


115it [04:29,  2.80s/it]


116it [04:32,  2.85s/it]


117it [04:34,  2.75s/it]


118it [04:36,  2.45s/it]


119it [04:39,  2.55s/it]


120it [04:42,  2.72s/it]


121it [04:45,  2.80s/it]


122it [04:47,  2.69s/it]


123it [04:49,  2.45s/it]


124it [04:52,  2.54s/it]


125it [04:54,  2.35s/it]


126it [04:57,  2.44s/it]


127it [05:00,  2.69s/it]


128it [05:02,  2.57s/it]


129it [05:04,  2.52s/it]


130it [05:06,  2.34s/it]


131it [05:10,  2.67s/it]


132it [05:13,  2.87s/it]


133it [05:16,  2.91s/it]


134it [05:19,  2.74s/it]


135it [05:21,  2.58s/it]


136it [05:24,  2.79s/it]


137it [05:25,  2.32s/it]


138it [05:27,  2.19s/it]


...


Recall = 0.6656391133824575
Precision = 0.6742632979425822





602it [23:37,  2.40s/it]


603it [23:40,  2.37s/it]


604it [23:42,  2.44s/it]


605it [23:44,  2.35s/it]


606it [23:46,  2.13s/it]


607it [23:49,  2.44s/it]


608it [23:51,  2.31s/it]


609it [23:54,  2.44s/it]


610it [23:56,  2.36s/it]


611it [23:59,  2.41s/it]


612it [24:00,  2.03s/it]


613it [24:03,  2.26s/it]


614it [24:04,  2.13s/it]


615it [24:07,  2.23s/it]


616it [24:09,  2.28s/it]


617it [24:11,  2.08s/it]


618it [24:13,  2.04s/it]


619it [24:15,  2.08s/it]


620it [24:17,  1.95s/it]


621it [24:19,  2.12s/it]


622it [24:20,  1.87s/it]


623it [24:22,  1.78s/it]


624it [24:24,  1.89s/it]


625it [24:27,  2.16s/it]


626it [24:29,  1.99s/it]


627it [24:30,  1.97s/it]


628it [24:32,  1.85s/it]


629it [24:34,  1.90s/it]


630it [24:36,  1.82s/it]


631it [24:38,  1.84s/it]


632it [24:40,  2.05s/it]


633it [24:42,  2.05s/it]


634it [24:45,  2.27s/it]


635it [24:47,  2.21s/it]


636it [24:50,  2.46s/it]


637it [24:52,  2.34s/it]


638it [24:54,  2.10s/it]



Recall = 0.6672205098159089
Precision = 0.6752236742295582





702it [27:14,  2.03s/it]


703it [27:16,  2.18s/it]


704it [27:19,  2.39s/it]


705it [27:22,  2.44s/it]


706it [27:23,  2.20s/it]


707it [27:25,  2.02s/it]


708it [27:26,  1.87s/it]


709it [27:28,  1.93s/it]


710it [27:32,  2.41s/it]


711it [27:34,  2.29s/it]


712it [27:35,  2.08s/it]


713it [27:38,  2.09s/it]


714it [27:39,  1.96s/it]


715it [27:42,  2.10s/it]


716it [27:44,  2.19s/it]


717it [27:46,  2.20s/it]


718it [27:48,  1.99s/it]


719it [27:50,  2.07s/it]


720it [27:53,  2.23s/it]


721it [27:55,  2.16s/it]


722it [27:57,  2.29s/it]


723it [27:59,  2.09s/it]


724it [28:01,  2.03s/it]


725it [28:02,  1.90s/it]


726it [28:05,  2.00s/it]


727it [28:07,  2.27s/it]


728it [28:10,  2.20s/it]


729it [28:12,  2.26s/it]


730it [28:13,  1.94s/it]


731it [28:15,  1.81s/it]


732it [28:17,  2.02s/it]


733it [28:20,  2.25s/it]


734it [28:22,  2.28s/it]


735it [28:24,  2.21s/it]


736it [28:28,  2.65s/it]


737it [28:30,  2.43s/it]


738it [28:32,  2.44s/it]


Final:
Recall = 0.6679747828185325
Precision = 0.6757250777563274
